# Time-frequency analysis of EEG signals

author: Carina Forster

contact: forster@cbs.mpg.de

last updated 01.07.2024

Aim: what is the spectral content of the data?

In [1]:
import mne
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

In [2]:
%matplotlib qt

In [5]:
# load liverpool data (preprocessed)
epochs = mne.read_epochs(Path(data_dir, 'liverpool_fpz_reference-epo.fif'))

# concatenate epochs back to raw
raw = mne.concatenate_epochs([epochs])
raw.info

Reading C:\Users\Carina\Desktop\data_liverpool\liverpool_fpz_reference-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    3000.00 ms
        0 CTF compensation matrices available
Not setting metadata
188 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
188 matching events found
No baseline correction applied


<Info | 11 non-empty values
 bads: []
 ch_names: Fp1, AF7, AF3, F1, F3, F5, F7, FT7, FC5, FC3, FC1, C1, C3, C5, ...
 chs: 64 EEG, 2 EOG, 2 ECG, 1 Stimulus
 custom_ref_applied: True
 dig: 67 items (3 Cardinal, 64 EEG)
 file_id: 4 items (dict)
 highpass: 0.1 Hz
 lowpass: 30.0 Hz
 meas_date: 2024-06-13 14:42:10 UTC
 meas_id: 4 items (dict)
 nchan: 69
 projs: []
 sfreq: 512.0 Hz
>

Let's look at power

In [6]:
# why do we drop Fcz?
spectrum_epochs = epochs.drop_channels(['Fpz']).compute_psd(fmin=2, fmax=40)
spectrum_raw = raw.drop_channels(['Fpz']).compute_psd(fmin=2, fmax=40)

    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows


In [7]:
len(spectrum_raw.freqs)

114

In [8]:
spectrum_epochs.plot(dB=True);

Plotting power spectral density (dB=True).


Averaging across epochs...


C:\Users\Carina\AppData\Local\Temp\ipykernel_3100\1468739425.py:1: FutureWarning: The value of `amplitude='auto'` will be removed in MNE 1.8.0, and the new default will be `amplitude=False`.
  spectrum_epochs.plot(dB=True);


In [9]:
spectrum_epochs.plot(dB=False);

Plotting amplitude spectral density (dB=False).
Averaging across epochs...


C:\Users\Carina\AppData\Local\Temp\ipykernel_3100\3531536814.py:1: FutureWarning: The value of `amplitude='auto'` will be removed in MNE 1.8.0, and the new default will be `amplitude=False`.
  spectrum_epochs.plot(dB=False);


In [11]:
spectrum_epochs.plot_topomap(ch_type='eeg', normalize=True, dB=True);

<div class="alert alert-block alert-success">
<b>Exercise:</b> 

Determine the individual peaks from the PSD

</div>

In [19]:
spectrum = spectrum_epochs

import scipy
import numpy as np

spectrum._data.shape

# so we average over channels and epochs
spectrum_avg = np.mean(spectrum._data, axis=(0,1))

vis_index = epochs.ch_names.index('O2')

# let's pick channel O2
spectrum_vis = np.mean(spectrum._data[:, vis_index, :], axis=0)

peak = scipy.signal.find_peaks(spectrum_avg)
peak_vis = scipy.signal.find_peaks(spectrum_vis)

In [20]:
# where are the peaks?
for p in peak[0]:
    print(spectrum.freqs[p])

for p in peak_vis[0]:
    print(spectrum.freqs[p])

6.662329212752115
14.324007807417047
22.318802862719586
24.31750162654522
26.982433311646066
28.31489915419649
6.662329212752115
14.657124268054654
22.318802862719586
26.982433311646066
27.648666232921276
28.9811320754717


In [23]:
# plot the spectrum with the peaks highlighted
plt.plot(spectrum.freqs, spectrum_avg)
for p in peak[0]:
    plt.plot(spectrum.freqs[p], spectrum_avg[p], 'ro')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power Spectral Density (dB)')
plt.show()

In [22]:
# plot the spectrum with the peaks highlighted for the visual channel
plt.plot(spectrum.freqs, spectrum_vis)
for p in peak_vis[0]:
    plt.plot(spectrum.freqs[p], spectrum_vis[p], 'ro')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power Spectral Density (dB)')
plt.show()

Bonus:

load the resting state data and determine individual alpha peak frequency

check out:

https://mne.discourse.group/t/individual-alpha-frequency/1500/2

## But what about time? 

In [37]:
# load the clean epochs
data_dir = Path("C:/", "Users", "Carina", "Desktop", "data_liverpool")
epochs = mne.read_epochs(Path(data_dir, 'average_ref-epo.fif'))

Reading C:\Users\Carina\Desktop\data_liverpool\average_ref-epo.fif ...
    Found the data of interest:
        t =   -1000.64 ...    1000.64 ms
        0 CTF compensation matrices available
Not setting metadata
320 matching events found
No baseline correction applied
0 projection items activated


Let's extract only epochs that contained a visual stimulus

In [50]:
visual_epochs = epochs['LV', 'RV']
visual_epochs

Number of events,144
Events,LV: 73RV: 71
Time range,-1.001 – 1.001 s
Baseline,-1.001 – 1.001 s


In [51]:
freqs = np.logspace(*np.log10([6, 35]), num=16) # frequency resolution
n_cycles = freqs / 4.0  # different number of cycle per frequency


power, itc = visual_epochs.compute_tfr(
    method="morlet",
    freqs=freqs,
    n_cycles=n_cycles,
    average=True,
    return_itc=True,
    decim=3,
)

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    2.0s


In [52]:
freqs

array([ 6.        ,  6.74857953,  7.59055427,  8.53757652,  9.60275235,
       10.80082298, 12.14836881, 13.66403883, 15.36880878, 17.28627138,
       19.44296285, 21.86873017, 24.59714411, 27.66596385, 31.11765953,
       35.        ])

In [53]:
n_cycles

array([1.5       , 1.68714488, 1.89763857, 2.13439413, 2.40068809,
       2.70020575, 3.0370922 , 3.41600971, 3.8422022 , 4.32156784,
       4.86074071, 5.46718254, 6.14928603, 6.91649096, 7.77941488,
       8.75      ])

In [68]:
# let's pick a visual channel
channel = ['EEG 058']

In [70]:
power.copy().crop(-0.5,0.5).plot(picks=channel, title=f'Power in channel {channel[0]}', mode="mean")

No baseline correction applied


[<Figure size 640x480 with 2 Axes>]

In [78]:
power.copy().crop(-0.5, 0.5).plot(picks=channel, title=f'Power in channel {channel[0]}', baseline=(-0.5, 0), mode="percent")

Applying baseline correction (mode: percent)


[<Figure size 640x480 with 2 Axes>]

In [77]:
power.copy().crop(-0.5, 0.5).plot(picks=channel, title=f'Power in channel {channel[0]}', baseline=(-0.5, 0), mode="zscore")

Applying baseline correction (mode: zscore)


[<Figure size 640x480 with 2 Axes>]

In [80]:
fig, axes = plt.subplots(1, 2, figsize=(7, 4), layout="constrained")
topomap_kw = dict(
    ch_type="eeg", show=False
)
plot_dict = dict(Alpha=dict(fmin=8, fmax=12), Beta=dict(fmin=13, fmax=25))
for ax, (title, fmin_fmax) in zip(axes, plot_dict.items()):
    power.plot_topomap(**fmin_fmax, axes=ax, **topomap_kw, show_names=False)
    ax.set_title(title)

No baseline correction applied
No baseline correction applied


In [81]:
power.plot_joint();

No baseline correction applied
No baseline correction applied


Bonus: [Evoked vs. induced signals](https://www.fil.ion.ucl.ac.uk/~karl/Mechanisms%20of%20evoked%20and%20induced%20responses.pdf)

In [96]:
visual_epochs.average().plot(picks=channel)
power.crop(-0.5,0.5).plot(picks=channel, baseline=(-0.5, 0), mode="zscore")

visual_epochs.subtract_evoked().average().plot(picks=channel)

visual_epochs_induced = visual_epochs.copy().subtract_evoked()

freqs = np.logspace(*np.log10([6, 35]), num=16) # frequency resolution
n_cycles = freqs / 4.0  # different number of cycle per frequency

power_induced, _ = visual_epochs_induced.compute_tfr(
    method="morlet",
    freqs=freqs,
    n_cycles=n_cycles,
    average=True,
    return_itc=True,
    decim=3,
)


Need more than one channel to make topography for eeg. Disabling interactivity.


C:\Users\Carina\AppData\Local\Temp\ipykernel_3100\2893891377.py:5: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.time_frequency.tfr.AverageTFR'>.tmax (0.497823 s)
  power.crop(-0.5,0.5).plot(picks=channel, baseline=(-0.5, 0), mode="zscore")


IndexError: boolean index did not match indexed array along axis 2; size of axis is 401 but size of corresponding boolean axis is 201

In [93]:
# Is the power evoked or induced?

power_induced.crop(-0.5,0.5).plot(picks=channel, baseline=(-0.5, 0), mode="zscore")
power.crop(-0.5,0.5).plot(picks=channel, baseline=(-0.5, 0), mode="zscore")

ValueError: operands could not be broadcast together with shapes (59,16,401) (59,16,201) 

<div class="alert alert-block alert-warning">
<b>Discussion:</b> 

When do you choose PSD over TFR?

</div>


<div class="alert alert-block alert-info">
<b>Bonus:</b> 

Wavelets excurs:

In the context of Morlet wavelets, the relationship between the number of cycles and temporal precision is an important aspect of time-frequency analysis. Here’s a detailed explanation:

Morlet Wavelets
A Morlet wavelet is a complex sinusoid modulated by a Gaussian window. It is widely used in time-frequency analysis because it provides a good balance between time and frequency resolution.

Key Concepts
Number of Cycles: This refers to how many oscillations of the sinusoidal part of the wavelet fit within the Gaussian window.
Temporal Precision: This is how precisely the wavelet can localize events in time.
Frequency Precision: This is how precisely the wavelet can localize events in frequency.
Relationship Between Cycles and Precision
Low Number of Cycles:
High Temporal Precision: The wavelet is shorter in duration, allowing it to better localize transient events in time.
Low Frequency Precision: The wavelet has a broader frequency response, meaning it is less precise in distinguishing between different frequencies.
High Number of Cycles:
Low Temporal Precision: The wavelet is longer in duration, which means it cannot localize transient events in time as well.
High Frequency Precision: The wavelet has a narrower frequency response, providing better resolution of different frequencies.
Trade-Off
There is an inherent trade-off between temporal and frequency precision when using Morlet wavelets:

Short Wavelets (few cycles): Good for time localization, but poor for frequency localization.
Long Wavelets (many cycles): Good for frequency localization, but poor for time localization.
Practical Implications
In practical applications, the choice of the number of cycles depends on the specific requirements of the analysis:

Analyzing Rapid Transients: Use fewer cycles to achieve better temporal resolution.
Analyzing Steady-State Oscillations: Use more cycles to achieve better frequency resolution.
Example
Suppose you are analyzing EEG data to study brain rhythms:

If you are interested in detecting short, transient events like spikes, you would use a wavelet with fewer cycles (e.g., 3-5 cycles) to get high temporal precision.
If you are interested in the detailed spectral content of ongoing oscillations, you would use a wavelet with more cycles (e.g., 7-10 cycles) to get high frequency precision.

</div>